<a href="https://colab.research.google.com/github/Osinskaya/projects/blob/main/%D0%98%D1%81%D1%81%D0%BB%D0%B5%D0%B4%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BD%D0%B0%D0%B4%D1%91%D0%B6%D0%BD%D0%BE%D1%81%D1%82%D0%B8_%D0%B7%D0%B0%D1%91%D0%BC%D1%89%D0%B8%D0%BA%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.


Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

In [ ]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()


In [ ]:
data = pd.read_csv('/datasets/data.csv')

In [ ]:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Проверим связаны ли между собой пропуски:**

In [ ]:
data[data['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [ ]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [ ]:
data['days_employed'].max() / 365

1100.6997273296713

### Замечания по данным:

1. Большое количество отрицательных значения в столбце 'days_employed';


2. Разный регистр в столбце 'education';


3. Пропуски в столбцах 'days_employed' и 'total_income';


4. Максимально значение столбца 'days_employed' 401755 = 1100 лет;


5. Значения -1 и 20 в столбце 'children';


6. Минимальный возраст 0 лет.

**Посчитаем количество строк со значениями -1 и 20 в стоблце 'children':**

In [ ]:
data[data['children'] == -1]['children'].count()

47

In [ ]:
data[data['children'] == 20]['children'].count()

76

**Посчитаем количество людей с 0 возрастом:**

In [ ]:
data[data['dob_years'] == 0]['dob_years'].count()

101

*101 значение где возраст равен 0*

### Вывод

* В данных присутвуют пропуски в столбцах 'days_employed' и 'total_income'. Причем пропусков одинакове количество *(возможно связано с тем, что человек не имеющий опыта работы и следовательно не имеющий ежемесячной запрлаты пропускал эти поля)*; 


* Необходимо привести все данные в столбце 'education' к одному регистру;


* Непонятно почему в столбце 'days_employed' стоят отрицательные значения;


* В столбце 'days_employed' найден артефакт со стажем работы 1100 лет *(скорее всего это опечатка при вводе данных)*;


* В столбце 'children' найдено 47 значений = '-1' и 76 значений = '20' *(-1 похоже на опечатку при вводе данных, а 20(???))*;


* В столбце 'dob_years' найдено 101 значение = '0' *(возможно 0 стоят вместо пропусков)*.

### Шаг 2. Предобработка данных

### Обработка пропусков

***Обработка пропусков в столбце 'total_income'.***

In [ ]:
#Для начала посмотрим у заемщиков с каким типом занятости есть пропуски в ежемесячной зарплате.

data.loc[data['total_income'].isna(), 'income_type'].value_counts() 


сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

In [ ]:
#Создадим таблицу в которой сохраним тип занятости
#и медианное значение ежемесячной зарплаты для определнного типа занятости.

median_total_income = data.groupby('income_type')['total_income'].median()
median_total_income


income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

In [ ]:
#Заполним пропуски медианными значениями 
#в соответсвии с типом занятости.

data.loc[(data['total_income'].isna())&(data['income_type'] == 'сотрудник'), 'total_income'] = median_total_income[6]
data.loc[(data['total_income'].isna())&(data['income_type'] == 'компаньон'), 'total_income'] = median_total_income[3]
data.loc[(data['total_income'].isna())&(data['income_type'] == 'пенсионер'), 'total_income'] = median_total_income[4]
data.loc[(data['total_income'].isna())&(data['income_type'] == 'госслужащий'), 'total_income'] = median_total_income[2]
data.loc[(data['total_income'].isna())&(data['income_type'] == 'предприниматель'), 'total_income'] = median_total_income[5]


In [ ]:
#Проверим сколько осталось пропусков в столбце 'total_income'.

data['total_income'].isna().sum()

0

***Обработка нулевых значения в столбце 'dob_years':***

In [ ]:
#Создадим таблицу в которой сохраним медианное значение возраста
#для каждого типа занятости.

median_dob_years = data.groupby('income_type')['dob_years'].median()
median_dob_years

income_type
безработный        38.0
в декрете          39.0
госслужащий        40.0
компаньон          39.0
пенсионер          60.0
предприниматель    42.5
сотрудник          39.0
студент            22.0
Name: dob_years, dtype: float64

In [ ]:
#Выведем типы занятости в которых присутсвуют заемщики с нулевым возрастом.

data.loc[data['dob_years'] == 0, 'income_type'].value_counts()

сотрудник      55
пенсионер      20
компаньон      20
госслужащий     6
Name: income_type, dtype: int64

In [ ]:
#Вместо нулевых значений в столбце 'dob_years' подставим
#медианное значение возраста для определенного типа занятости.

data.loc[(data['dob_years'] == 0)&(data['income_type'] == 'сотрудник'), 'dob_years'] = median_dob_years[6]
data.loc[(data['dob_years'] == 0)&(data['income_type'] == 'компаньон'), 'dob_years'] = median_dob_years[3]
data.loc[(data['dob_years'] == 0)&(data['income_type'] == 'пенсионер'), 'dob_years'] = median_dob_years[4]
data.loc[(data['dob_years'] == 0)&(data['income_type'] == 'госслужащий'), 'dob_years'] = median_dob_years[2]



In [ ]:
#Проверим на нулевой возраст.

data.loc[data['dob_years'] == 0, 'income_type'].count()

0

In [ ]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


***Обработка пропусков в столбце 'days_employed':***

In [ ]:
#Избавимся от отрицательных значений, взяв все значения по модулую.

data['days_employed'] = data['days_employed'].abs()

In [ ]:
#Посмотрим общие сведения по возрасту заемщиков.

data['dob_years'].describe()

count    21525.000000
mean        43.496167
std         12.231538
min         19.000000
25%         34.000000
50%         43.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

In [ ]:
#Выделим четыре группы: молодые(от 18 до 28), зрелые(от 29 до 43), взрослые(от 44 до 54), пожилые(55+).
#Для этого напишем функцию, которая на вход принимает возраст заемщика и возвращает его возрастную группу.

def age_group(dob_years):
    if dob_years <= 28:
        return 'молодой'
    elif dob_years <= 43:
        return 'зрелый'
    elif dob_years <= 54:
        return 'взрослый'
    else:
        return 'пожилой'
    
data['age_group'] = data['dob_years'].apply(age_group)
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,зрелый
1,1,4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,зрелый
2,0,5623.422610,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,зрелый
3,3,4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,зрелый
4,0,340266.072047,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,взрослый
5,0,926.185831,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,молодой
6,0,2879.202052,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,зрелый
7,0,152.779569,50.0,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,взрослый
8,2,6929.865299,35.0,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,зрелый
9,0,2188.756445,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,зрелый


In [ ]:
#Создадим таблицу с медианным значением трудового стажа для каждой возрастной группы.

median_days_employed = data.groupby('age_group')['days_employed'].median()
median_days_employed

age_group
взрослый      2567.889652
зрелый        1648.414805
молодой        941.880042
пожилой     345954.616737
Name: days_employed, dtype: float64

In [ ]:
#Заполним пропуски в столбце 'days_employed' медианным значением для каждой возрастной группы соотвествено.

data.loc[(data['days_employed'].isna())&(data['age_group'] == 'взрослый'), 'days_employed'] = median_days_employed[0]
data.loc[(data['days_employed'].isna())&(data['age_group'] == 'зрелый'), 'days_employed'] = median_days_employed[1]
data.loc[(data['days_employed'].isna())&(data['age_group'] == 'молодой'), 'days_employed'] = median_days_employed[2]
data.loc[(data['days_employed'].isna())&(data['age_group'] == 'пожилой'), 'days_employed'] = median_days_employed[3]

In [ ]:
#Проверим сколько пропусков осталось в столбце 'days_employed'.

data['days_employed'].isna().sum()

0

### Вывод

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
age_group           21525 non-null object
dtypes: float64(3), int64(4), object(6)
memory usage: 2.1+ MB


Как мы можем увидеть, все пропуски заполнены.
* Для заполнения пропусков в столбце 'total_income'  нашли медианное значение з/п для каждого типа занятости и подставили его в пропуски.


* Для избавления от нулевого возраста заменили его на медианное значение возраста для каждого типа занятости.


* Для заполнения пропусков в столбце 'days_employed'  разделили всех заемщиков на 3 возрастные группы и заполнили пропуски медианным значением стажа работы для каждой возрастной группы.

### Замена типа данных

In [ ]:
#Переведем дни рабочего стажа в года и приведем их к целочисленному типу данных.

data['days_employed'] = data['days_employed'] / 365
data['days_employed'] = data['days_employed'].astype('int')

In [ ]:
#Переведем ежемесячную зарплату из вещественного в целочисленный тип данных.

data['total_income'] = data['total_income'].astype('int')

In [ ]:
#Переведем возраст заемщиков из вещественного в целочисленный тип данных.


data['dob_years'] = data['dob_years'].astype('int')

### Вывод

Нам ни к чему вещественный тип данных в столбцах 'days_employed', 'total_income' и 'dob_years', поэтому для удобства восприятия приведем их к целочисленному типу используя метод .astype(), мы можем его использоват так как ранее избавились от всех NaN.

### Обработка дубликатов

In [ ]:
#Проверим данные на дубликаты.

data.duplicated().sum()

54

In [ ]:
#Ранее в столбце 'education' были замечено написание образование в разном регистре,
#приведем все к одному регистру, чтобы точно найти все дубликаты.

data['education'] = data['education'].str.lower()
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,зрелый
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,зрелый
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,зрелый
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,зрелый
4,0,932,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,взрослый


In [ ]:
data['days_employed'].describe()

count    21525.000000
mean       187.294402
std        383.123365
min          0.000000
25%          2.000000
50%          5.000000
75%         15.000000
max       1100.000000
Name: days_employed, dtype: float64

In [ ]:
#Проверим данные на дубликаты после приведения столбца 'education' к одному регистру.
data.duplicated().sum()

71

In [ ]:
#Удалим дубликаты.

data = data.drop_duplicates().reset_index(drop = True)

#Проверим на наличие дубликатов после удаления.

data.duplicated().sum()

0

### Вывод

Из-за разного регистра в столбце 'education' методом .duplicated() находились не все дубликаты. После приведения столбца в нормальный вид, можно удалять все дубликаты методом .drop_duplicates().

### Лемматизация

In [ ]:
#Посмотрим все варианты целей кредита.

data['purpose'].value_counts()


свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

*Многие варианты подразумевают одно и тоже, но записаны по-разному.*

In [ ]:
#Выделим основные категории целей кредита.

categories = ['свадьба','жилье','автомобиль','образование','недвижимость']

#Напишем функцию, которая будет принимать на вход цель кредита, а возвращать котегорию кредита.

def lemmatize(text):
    lemmas = m.lemmatize(text)
    for word in categories:
        if word in lemmas:
            lemma = word
    return lemma

data['purpose_categories'] = data['purpose'].apply(lemmatize)        
data.head()
        

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_categories
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,зрелый,жилье
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,зрелый,автомобиль
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,зрелый,жилье
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,зрелый,образование
4,0,932,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,взрослый,свадьба


In [ ]:
#Посмотрим для каких целей чаще всего берут кредит.

data['purpose_categories'].value_counts()

недвижимость    6351
жилье           4460
автомобиль      4306
образование     4013
свадьба         2324
Name: purpose_categories, dtype: int64

In [ ]:
#Объединим категории 'недвижимость' и 'жилье'.

data.loc[data['purpose_categories'] == 'жилье', 'purpose_categories'] = 'недвижимость'

In [ ]:
data['purpose_categories'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_categories, dtype: int64

### Вывод

In [ ]:
data['purpose_categories'].value_counts(normalize = True)

недвижимость    0.503915
автомобиль      0.200708
образование     0.187051
свадьба         0.108325
Name: purpose_categories, dtype: float64

Для упрощения дальнейшего анализа данных мы выделили основные ключевые слова во всех вариантах написания целей и на их основании создали словарь категорий: свадьба, жилье, автомобиль, образование, недвижимость. С помощью функции лематизации создали столбец, где вместо свободного написания цели кредита записана определнная категория.
После такого преобразования данные легко категоризировать и анализировать для каких целей чаще всего берут кредиты, в нашем случае самой популярной целью взятия кредита (50%) является - недвижимость.

### Категоризация данных

**Ранее мы уже категоризировали возраст заемщиков и цели кредита.**

In [ ]:
data['age_group'].value_counts(normalize = True).round(2)

зрелый      0.40
взрослый    0.25
пожилой     0.22
молодой     0.12
Name: age_group, dtype: float64

In [ ]:
data['purpose_categories'].value_counts(normalize = True).round(2)

недвижимость    0.50
автомобиль      0.20
образование     0.19
свадьба         0.11
Name: purpose_categories, dtype: float64

**Категоризируем заемщиков по количеству детей:**

In [ ]:
#Заменим значение -1 на 1

data.loc[data['children'] == -1 , 'children'] = 1

In [ ]:
#Заменим значение 20 на 2

data.loc[data['children'] == 20, 'children'] = 2

In [ ]:
#Присвоим заемщикам у которых нет детей группу 0.
#Заемщикам у которых есть дети группу 1.


def children_group(children):
    if children > 0:
        return 'с детьми'
    else:
        return 'без детей'

data['children_group'] = data['children'].apply(children_group)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_categories,children_group
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,зрелый,недвижимость,с детьми
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,зрелый,автомобиль,с детьми
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,зрелый,недвижимость,без детей
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,зрелый,образование,с детьми
4,0,932,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,взрослый,свадьба,без детей


In [ ]:
data['children_group'].value_counts(normalize = True).round(2)

без детей    0.66
с детьми     0.34
Name: children_group, dtype: float64

**Осталось категоризировать доход.**

In [ ]:
statistics = data['total_income'].describe()
statistics[3:9]

min      20667.00
25%     107623.00
50%     142594.00
75%     195820.25
max    2265604.00
Name: total_income, dtype: float64

In [ ]:
low_income = data['total_income'].quantile(0.25)
low_income

107623.0

In [ ]:
average_income = data['total_income'].quantile(0.5)
average_income

142594.0

In [ ]:
hight_income = data['total_income'].quantile(0.75)     
hight_income

195820.25

In [ ]:
def group_icome(income):
    if income < low_income:
        return 'низкий доход'
    elif (income >=  low_income) and (income < average_income):
        return 'средний доход'
    elif (income >= average_income) and (income < hight_income):
        return 'доход выше среднего'
    else:
        return 'высокий доход'
    
data['income_group'] = data['total_income'].apply(group_icome)        
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_categories,children_group,income_group
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,зрелый,недвижимость,с детьми,высокий доход
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,зрелый,автомобиль,с детьми,средний доход
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,зрелый,недвижимость,без детей,доход выше среднего
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,зрелый,образование,с детьми,высокий доход
4,0,932,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,взрослый,свадьба,без детей,доход выше среднего


In [ ]:
data['income_group'].value_counts(normalize = True).round(2)

доход выше среднего    0.29
низкий доход           0.25
высокий доход          0.25
средний доход          0.21
Name: income_group, dtype: float64

### Вывод

* Категоризация возраста заемщиков происходила делением на четыре основные группы: ***молодые(от 18 до 28), зрелые(от 29 до 43), взрослые(от 44 до 54), пожилые(55+)***, при помощи функции age_group, которая была описана при обработке пропусков в столбце 'days_employed'.
*Доля взрослых составила 50%, доля молодых и пожилых по 25%.*  


* При категоризация целей кредита были изучены все варианты их написания и выделены 5 основных категорий: ***свадьба***, ***жилье***, ***автомобиль***, ***образование***, ***недвижимость***. При помощи фукции lemmatize, которая выделяет леммы в свободном варианте написания цели и присваивает ей определнную категорию, всем вариантам написания целей была присвоена категория цели займа. После чего, для удобства, категория "жилье" переименова в категорию "недвижимость", так как они по сути являются одинаковыми.
*Доля кредитов на недвижимость составила 50%, на автомобиль 20%, на образование и свадьбу 19% и 11% соотвествено.*  


* При категоризации заемщиков по количеству детей выделили две группы: ***0 - без детей***, ***1 - есть дети***.
*Доля заемщиков без детей составила 66%, c детьми 34%*  


* При категоризации дохода с помощью квантилей были выделены 4 основные группы: ***низкий доход*** < 107623, ***средний доход*** от 107623  до 142594, ***доход выше среднего***  от 142594 до 195820.25 и ***высокий доход*** > 195820.25. 
*Доля дохода выше среднего составила 29%, нижзкий и высокий доход 25%, доля среднего дохода составила 21%.*

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
#Сформируем сводную таблицу с наличием детей и задолженностью по кредиту.

data_pivot = data.pivot_table(index = ['children_group'], values = 'debt', aggfunc= ['count','sum','mean'] ).round(3)
data_pivot

,count,sum,mean
,debt,debt,debt
children_group,,,
без детей,14091,1063,0.075
с детьми,7363,678,0.092


### Вывод

Заемщики без детей в два раза чаще берут кредиты, при этом  заемщики с детьми выплачивают кредит вовремя на 1,7% реже заемщиков без детей. 

Возможно это связано с тем, что когда перед заемщиком стоит выбор между выплатой кредита и обеспечением необходимыми благами своего ребенка, он выбирает ребенка, тем самым просрачивая выплаты по кредиту.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
#Сформируем сводную таблицу с семейным положением и задолженностью по кредиту.

data_pivot = data.pivot_table(index = ['family_status'], columns = 'debt', values = 'gender', aggfunc = 'count')
data_pivot

debt,0,1
family_status,,
Не женат / не замужем,2536,274
в разводе,1110,85
вдовец / вдова,896,63
гражданский брак,3763,388
женат / замужем,11408,931


In [ ]:
#Посчитаем вероятность задолженности для каждого вида семейного положения.

data_pivot['ratio'] = data_pivot[1] / (data_pivot[0] + data_pivot[1])
data_pivot.sort_values('ratio', ascending = False)

debt,0,1,ratio
family_status,,,
Не женат / не замужем,2536,274,0.097509
гражданский брак,3763,388,0.093471
женат / замужем,11408,931,0.075452
в разводе,1110,85,0.071130
вдовец / вдова,896,63,0.065693


### Вывод

Заемщики не состоящие в официально офорленных отношениях реже выплачивают кредит вовремя.

Возможно это связано с тем, что находясь в браке ответсвенность за кредит чувсвуют оба супруга и выплачивают его вместе, тем самым повышая шанс своевременных выплат. Как оказалось, люди в браке более надежные заемщики для банка.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
#Сформируем сводную таблицу с уровнем дохода и задолженностью по кредиту.

data_pivot = data.pivot_table(index = ['income_group'], columns = 'debt', values = 'gender', aggfunc = 'count')
data_pivot

debt,0,1
income_group,,
высокий доход,4981,383
доход выше среднего,5771,546
низкий доход,4937,427
средний доход,4024,385


In [ ]:
#Посчитаем вероятность задолженности для каждого вида семейного положения.

data_pivot['ratio'] = data_pivot[1] / (data_pivot[0] + data_pivot[1])
data_pivot.sort_values('ratio', ascending = False)

debt,0,1,ratio
income_group,,,
средний доход,4024,385,0.087321
доход выше среднего,5771,546,0.086433
низкий доход,4937,427,0.079605
высокий доход,4981,383,0.071402


### Вывод

Неудивительно, что заемщики с высоким уровнем дохода (больше 195.820 тыс.руб.) чаще всего вовремя выплачивают кредит, вероятность того, что они просрочат выплаты состалвяет 7,1%.  
Что странно заемщики с низким уровнем дохода (менее 107.623 тыс. руб.) просрочат выплаты в 7,9% случаев. Это меньше, чем заемщики со средним и выше среднего доходом (от 142.594 до 195.820 тыс.руб.). Они просрочат выплаты с вероятностью 8,6% и 8,7% соответсвенно.  

Возможно более высокая вероятность своевременной выплаты кредита у людей с низким доходом связана с тем, что у них лучше развито умение рапоряжаться деньгами. А у людей со средним и выше среднего доходом потребности больше, а умение правильно распоряжаться деньгами и планировать свои траты меньше.

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
#Сформируем сводную таблицу с целью кредита и задолженностью по кредиту.

data_pivot = data.pivot_table(index = ['purpose_categories'], columns = 'debt', values = 'gender', aggfunc = 'count')
data_pivot

debt,0,1
purpose_categories,,
автомобиль,3903,403
недвижимость,10029,782
образование,3643,370
свадьба,2138,186


In [ ]:
#Посчитаем вероятность задолженности для каждого вида семейного положения.

data_pivot['ratio'] = data_pivot[1] / (data_pivot[0] + data_pivot[1])
data_pivot.sort_values('ratio', ascending = False)

debt,0,1,ratio
purpose_categories,,,
автомобиль,3903,403,0.093590
образование,3643,370,0.092200
свадьба,2138,186,0.080034
недвижимость,10029,782,0.072334


### Вывод

Заемщики берущие кредит на покупку автомобиля реже всего вовремя его выплачивают. А кредит взятый на недвижимость выплачивается лучше всего.

Возможно это связано с тем, что покупа недвижимости является серьезнее, чем покупа автомобиля, поэтому к выплатам по такому кредиту подходят ответсвеннее. 

### Шаг 4. Общий вывод

После исследования можно прийти к выводу, что наличие детей, семейное положение, уровень дохода и цель кредита влияют на его своевременное погашение следующи образом:

* вероятност задолженности по кредиту **при наличии детей** = 7,5%;
* вероятност задолженности по кредиту **без детей** = 9,2%;  


* вероятност задолженности по кредиту с семейным положением **не женат / не замужем** = 9,7%;
* вероятност задолженности по кредиту с семейным положением **гражданский брак** = 9,3%
* вероятност задолженности по кредиту с семейным положением **женат / замужем** = 7,5%;
* вероятност задолженности по кредиту с семейным положением **в разводе** =  7,1%;
* вероятност задолженности по кредиту с семейным положением **вдовец / вдова** = 6,6%;


* вероятност задолженности по кредиту со **средним уровнем дохода** = 8,7%;
* вероятност задолженности по кредиту с **доходом выше среднего** = 8,6;
* вероятност задолженности по кредиту с **низким уровнем доходом** = 8%;
* вероятност задолженности по кредиту с **высоким уровнем дохода** = 7,1%;


* вероятност задолженности по кредиту на **автомобиль** = 9,4%;
* вероятност задолженности по кредиту на **образование** = 9,2%;
* вероятност задолженности по кредиту на **свадьбу** = 8%;
* вероятност задолженности по кредиту на **недвижимость** = 7,2%.



Исходя из этого, можно сказать,что самым ответственным заемщиком является следующий человек: *состоящий или ранее состоявщий в официальном браке, без детей, с доходом свыше 195.820 тыс.руб, берущий кредит на недвижимость*  

А самым безответсвенным заемщиком является следующий человек: *не стостящий в официальном браке, с детьми, со средним доход, берущий кредит на автомобиль или образование.*